## Performance Analysis for Daily Energy Production

Caleb Phillips (caleb.phillips@nrel.gov)

The purpose of this notebook is to read in computed predictions and compare them to actual observations, computing metrics. 

In [ ]:
from common import *
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import re
from dw_tap.power_output import estimate_power_output
import os.path
import importlib
import power_output
import site_index
import plotly.express as px
plotly.offline.init_notebook_mode()

In [ ]:
index = site_index.SiteIndex()

### Load Ground Truth (Bergey Daily Data)

In [ ]:
daily_bergey = pd.read_csv("01 Bergey Turbine Data/daily_summaries.csv.bz2")
daily_bergey = daily_bergey.merge(index.index[['APRS ID','AID']].rename(columns={'APRS ID':'tid'}))
daily_bergey['date'] = pd.to_datetime(daily_bergey['date'],utc=True).dt.date
daily_bergey.head()

#### Evaluate faults

In [ ]:
pd.set_option('display.max_rows', 500)
daily_bergey[['soft_grid']].drop_duplicates().head(500)

In [ ]:
daily_bergey["fault"] = "None"
daily_bergey.loc[daily_bergey['faults'].str.contains("FAULT"),"fault"] = "Fault"
daily_bergey.loc[daily_bergey['faults'].str.contains("WAITING INITIALIZING"),"fault"] = "Reset"
daily_bergey.loc[daily_bergey['faults'].str.contains("MANUAL STOP"),"fault"] = "Stopped"
daily_bergey.loc[daily_bergey['soft_grid'].str.contains("%"),"fault"] = "Curtailed"
daily_bergey["fault"].value_counts()

### Load Prediction Data for all Models/Sites

In [ ]:
providers = ["bergey"]
models = ["perera","anl"]
wind_sources = ["wtk","wtk_bc","wtk_led_2018","wtk_led_2019","wtk_led_bc"]
obs_groups = ["all","bldgsonly","treesasbldgs"]

dfs = []

for provider in providers:
    for model in models:
        for wind_source in wind_sources:
            for og in obs_groups:
                for tid in index.tids(True):
                    fname = f"03 Model Outputs/{provider}_{model}_{tid}_{wind_source}_{og}.csv.bz2"
                
                    if not os.path.exists(fname):
                        continue

                    print(fname)

                    d = pd.read_csv(fname)
                    d["model"] = model
                    d["wind_source"] = wind_source
                    d["provider"] = provider
                    d["tid"] = tid
                    d["obsgroup"] = og
                    dfs.append(d[["model","obsgroup","wind_source","provider","tid","datetime","ws-adjusted"]])

                    if model == "perera":
                        d2 = d.copy()
                        d2["ws-adjusted"] = d2["ws-adjusted-2"]
                        d2["model"] = "shelter"
                        d2["obsgroup"] = og
                        dfs.append(d2[["model","obsgroup","wind_source","provider","tid","datetime","ws-adjusted"]])

                        d3 = d.copy()
                        d3["ws-adjusted"] = d3["ws-adjusted-3"]
                        d3["model"] = "shelter+"
                        d3["obsgroup"] = og
                        dfs.append(d3[["model","obsgroup","wind_source","provider","tid","datetime","ws-adjusted"]])

In [ ]:
# No model data
for provider in providers:
    for wind_source in tqdm(wind_sources):
        if wind_source == "wtk":
            d = pd.read_csv("01 Bergey Turbine Data/wtk.csv.bz2")
            d.rename(columns={"ws":"ws-adjusted"},inplace=True)
            
        elif wind_source == "wtk_led_2018":
            d = pd.read_csv("01 Bergey Turbine Data/wtk_led_2018.csv.bz2")
            d.rename(columns={"ws":"ws-adjusted","packet_date":"datetime"},inplace=True)
            
        elif wind_source == "wtk_led_2019":
            d = pd.read_csv("01 Bergey Turbine Data/wtk_led_2019.csv.bz2")
            d.rename(columns={"ws":"ws-adjusted","packet_date":"datetime"},inplace=True)
            
        elif wind_source == "wtk_bc":
            d = pd.read_csv("02 Bias Correction/wtk_bc.csv.bz2")
            d.rename(columns={"ws_bc":"ws-adjusted"},inplace=True)
            
        elif wind_source == "wtk_led_bc":
            d = pd.read_csv("02 Bias Correction/wtk_led_bc.csv.bz2")
            d.rename(columns={"ws_bc":"ws-adjusted"},inplace=True)
            
        else:
            print("Unsupported wind source")
            
        d["wind_source"] = wind_source
        d["model"] = "none"
        d["provider"] = provider
        d["obsgroup"] = "none"
        dfs.append(d[["model","obsgroup","wind_source","provider","tid","datetime","ws-adjusted"]])

In [ ]:
bigdf = pd.concat(dfs)
bigdf.head()

In [ ]:
bigdf["power_kw"] = power_output.Bergey10.windspeed_to_kw(bigdf,'ws-adjusted')
bigdf['datetime'] = pd.to_datetime(bigdf['datetime'],format="%Y-%m-%d %H:%M:%S",utc=True) # this is slow
bigdf['date'] = bigdf['datetime'].dt.date
bigdf.head()

In [ ]:
daily_summaries = bigdf[['model','obsgroup','wind_source','provider','tid','date','power_kw','ws-adjusted']]\
    .groupby(['model','obsgroup','wind_source','provider','tid','date']).agg({ 'power_kw': ['count','sum'], 
                                                                    'ws-adjusted': ['mean','max','min','median'] })
daily_summaries.columns = ['_'.join(col) for col in daily_summaries.columns.values]
daily_summaries = daily_summaries.reset_index()
# scale the 5 min data so that it is kwh (too)
daily_summaries["power_kw_sum"] = daily_summaries["power_kw_sum"]/(daily_summaries["power_kw_count"]/24)
daily_summaries.head()

### Create Merged Dataframe

In [ ]:
merged_df = daily_summaries.merge(daily_bergey[['date','tid','energy_kwh','fault']],on=['tid','date'])
merged_df = merged_df.dropna() # omit days where we are missing data
merged_df.head()

In [ ]:
# error percent compared to actual generation
merged_df["relative_error_pct"] = ((merged_df["power_kw_sum"] - merged_df["energy_kwh"])/merged_df["energy_kwh"])*100 
# error percent of max generation
merged_df["relative_error_pct_of_max"] = ((merged_df["power_kw_sum"] - merged_df["energy_kwh"])/300)*100 

merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['month'] = merged_df['date'].dt.month
merged_df['year'] = merged_df['date'].dt.year
merged_df['moyr'] = merged_df['date'].dt.year.astype(str)+"-"+merged_df['date'].dt.month.astype(str)

### Performance Plots: Overall

In [ ]:
filtered_merged_df = merged_df[(merged_df["fault"] == "None")]

In [ ]:
fig = px.scatter(filtered_merged_df[filtered_merged_df.obsgroup.isin(["treesasbldgs","none"])],
                 x="energy_kwh",y="power_kw_sum",
                 facet_row="model",facet_col="wind_source",
                 labels={"power_kw_sum":"Pred(kwh)","energy_kwh":"Obs(kwh)"})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.scatter(filtered_merged_df[filtered_merged_df.obsgroup.isin(["treesasbldgs","none"])],
                 x="energy_kwh",y="power_kw_sum",
                 facet_row="model",facet_col="wind_source",color="fault",
                 labels={"power_kw_sum":"Pred(kwh)","energy_kwh":"Obs(kwh)"})
fig.update_yaxes(matches=None)
fig.update_xaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.density_contour(filtered_merged_df[filtered_merged_df.obsgroup.isin(["treesasbldgs","none"])],
                         x="energy_kwh",y="power_kw_sum",
                         facet_row="model",facet_col="wind_source",
                         labels={"power_kw_sum":"Pred(kwh)","energy_kwh":"Obs(kwh)"})
fig.update_xaxes(range=[0, 40])
fig.update_yaxes(range=[0, 40])
fig.update_traces(contours_coloring="fill", contours_showlabels = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.box(filtered_merged_df,x="model",y="relative_error_pct_of_max",color="wind_source",
             labels={"relative_error_pct_of_max":"Relative Error (%)","model":"Model"},
             points=False,title="Daily Relative Error (% of Max Production)")
fig.show()

In [ ]:
fig = px.box(filtered_merged_df,x="wind_source",y="relative_error_pct_of_max",color="model",
             labels={"relative_error_pct_of_max":"Relative Error (%)","wind_source":"Wind Source"},
             points=False,title="Daily Relative Error (% of Max Production)")
fig.show()

In [ ]:
# Defining function for 50th Percentile
p25 = lambda x: x.quantile(0.25)
p75 = lambda x: x.quantile(0.75)
safemean = lambda x: np.mean([0 if np.isnan(y) else y for y in x])

daily_perf_summary = filtered_merged_df.groupby(["model","obsgroup","wind_source"]).\
    agg({ "relative_error_pct_of_max": ["mean","median",p25,p75] }).\
    rename(columns={"<lambda_0>":'25%ile',"<lambda_1>":'75%ile'})

daily_perf_summary

In [ ]:
daily_perf_summary.columns = ['_'.join(col) for col in daily_perf_summary.columns.values]
daily_perf_summary.reset_index(inplace=True)
daily_perf_summary.to_csv("04 Error Metrics/bergey_daily_relative_error.csv.bz2",index=False)

#### Daily Results By Site

In [ ]:
daily_bysite = filtered_merged_df.groupby(["model","obsgroup","wind_source","tid"]).\
    agg({ "relative_error_pct_of_max": ["mean","median",p25,p75] }).\
    rename(columns={"<lambda_0>":'25%ile',"<lambda_1>":'75%ile'})
daily_bysite.columns = ['_'.join(col) for col in daily_bysite.columns.values]
daily_bysite.reset_index(inplace=True)
daily_bysite.head()

In [ ]:
fig = px.bar(daily_bysite[daily_bysite['wind_source'].isin(["wtk","wtk_bc"])],x="tid",
             y="relative_error_pct_of_max_mean",color="model",facet_col="wind_source",
             facet_row="obsgroup",barmode='group',
             labels={"relative_error_pct_of_max_mean":"Relative Error (%)",
                     "tid":"Site","model":"Model"},
             title="Mean Relative Error in Daily Energy Estimate (% of kwh) by Model/Site")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.bar(daily_bysite[daily_bysite['wind_source'].isin(["wtk"]) & daily_bysite['model'].isin(["anl","none"])],x="tid",
             y="relative_error_pct_of_max_mean",color="obsgroup",barmode='group',
             labels={"relative_error_pct_of_max_mean":"Relative Error (%)",
                     "tid":"Site","model":"Model","obsgroup":"Obstacles"},
             title="Mean Relative Error in Daily Energy Estimate (% of kwh) by Model/Site")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.box(filtered_merged_df[filtered_merged_df['wind_source'].isin(["wtk","wtk_bc"])],x="tid",
             y="relative_error_pct_of_max",color="model",facet_row="wind_source",facet_col="obsgroup",
             labels={"relative_error_pct_of_max":"Relative Error (%)",
                     "tid":"Site","model":"Model"},points=False,
             title="Mean Relative Error in Daily Energy Estimate (% of kwh) by Model/Site")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.violin(filtered_merged_df[filtered_merged_df['wind_source'].isin(["wtk"]) & filtered_merged_df['model'].isin(["anl","none"])],
             y="tid",color='obsgroup',
             x="relative_error_pct_of_max",facet_row="wind_source",
             labels={"relative_error_pct_of_max":"Relative Error (%)",
                     "tid":"Site","obsgroup":"Obstacles"},points=False,
             width=700,height=1000,
             title="Mean Relative Error in Daily Energy Estimate (% of kwh) by Model/Site (WTK Wind Source)")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.histogram(filtered_merged_df[filtered_merged_df['wind_source'].isin(["wtk"]) & filtered_merged_df['model'].isin(["anl","none"])],
             x="power_kw_sum",color="obsgroup",facet_col="tid",
             labels={"power_kw_sum":"Energy (kwh)",
                     "tid":"Site","obsgroup":"Obstacles"},facet_col_wrap=4)
fig.update_yaxes(matches=None)
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()

In [ ]:
fig = px.box(filtered_merged_df[filtered_merged_df['wind_source'].isin(["wtk"]) & filtered_merged_df['model'].isin(["anl","none"])],
             x="power_kw_sum",color="obsgroup",y="tid",
             labels={"relative_error_pct_of_max":"Relative Error (%)",
                     "tid":"Site","obsgroup":"Obstacles",
                     "power_kw_sum":"Daily Energy Estimate (kwh)"},height=750,points=False)
fig.show()

#### Overall Results by Month and Site

In [ ]:
# Note this includes days with faults, so curtailment, etc. is included
bysitebymonth = merged_df.groupby(["model","obsgroup","wind_source","tid","moyr","month"]).\
    agg({ "power_kw_sum": "sum", "energy_kwh": "sum" })
#bysitebymonth.columns = ['_'.join(col) for col in bysitebymonth.columns.values]
bysitebymonth.reset_index(inplace=True)
# Using 300kwh * 30 days for maximum production in calculation of relative error
bysitebymonth["relative_error_pct_of_max"] = ((bysitebymonth["power_kw_sum"] - bysitebymonth["energy_kwh"])/(300*30))*100 
bysitebymonth["relative_error_pct"] = ((bysitebymonth["power_kw_sum"] - bysitebymonth["energy_kwh"])/bysitebymonth["energy_kwh"])*100 
bysitebymonth

In [ ]:
fig = px.box(bysitebymonth[bysitebymonth['wind_source'].isin(["wtk","wtk_bc"]) & bysitebymonth['model'].isin(["anl","none"])],x="tid",
             y="relative_error_pct_of_max",color="obsgroup",facet_row="wind_source",
             labels={"relative_error_pct_of_max":"Relative Error (%)",
                     "tid":"Site","obsgroup":"Obstacles"})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.box(bysitebymonth[bysitebymonth['wind_source'].isin(["wtk","wtk_bc"]) & bysitebymonth['model'].isin(["perera","none"])],x="tid",
             y="relative_error_pct_of_max",color="obsgroup",facet_row="wind_source",
             labels={"relative_error_pct_of_max":"Relative Error (%)",
                     "tid":"Site","obsgroup":"Obstacles"})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
p25 = lambda x: x.quantile(0.25)
p75 = lambda x: x.quantile(0.75)
safemean = lambda x: np.mean([0 if np.isnan(y) else y for y in x])

monthly_perf_summary = bysitebymonth[['model','wind_source','obsgroup','relative_error_pct_of_max']].groupby(['model','wind_source','obsgroup']).\
    agg({ "relative_error_pct_of_max": ["mean","median",p25,p75] }).\
    rename(columns={"<lambda_0>":'25%ile',"<lambda_1>":'75%ile'})

monthly_perf_summary

In [ ]:
monthly_perf_summary.columns = ['_'.join(col) for col in monthly_perf_summary.columns.values]
monthly_perf_summary.reset_index(inplace=True)
monthly_perf_summary.to_csv("04 Error Metrics/bergey_monthly_relative_error.csv.bz2",index=False)

#### Overall Results by Year

In [ ]:
# Note this includes days with faults, so curtailment, etc. is included
bysitebyyear = merged_df.groupby(["model","obsgroup","wind_source","tid","year"]).\
    agg({ "power_kw_sum": "sum", "energy_kwh": "sum" })
#bysitebymonth.columns = ['_'.join(col) for col in bysitebymonth.columns.values]
bysitebyyear.reset_index(inplace=True)
# Using 300kwh * 30 days for maximum production in calculation of relative error
bysitebyyear["relative_error_pct_of_max"] = ((bysitebyyear["power_kw_sum"] - bysitebyyear["energy_kwh"])/(300*30))*100 
bysitebyyear["relative_error_pct"] = ((bysitebyyear["power_kw_sum"] - bysitebyyear["energy_kwh"])/bysitebyyear["energy_kwh"])*100 
bysitebyyear

In [ ]:
fig = px.box(bysitebyyear,y="relative_error_pct_of_max",x="wind_source",color="model",facet_row="obsgroup",
            labels={"relative_error_pct_of_max":"Error(%)","wind_source":"Wind Source","model":"Model"})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.box(bysitebyyear[(bysitebyyear["wind_source"].isin(["wtk","wtk_bc"]))],y="relative_error_pct_of_max",x="wind_source",color="model",
       labels={"wind_source":"Wind Source","pct": "Annualized Error Percentage (%)","model":"Model"},
      facet_row="obsgroup")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
p25 = lambda x: x.quantile(0.25)
p75 = lambda x: x.quantile(0.75)
safemean = lambda x: np.mean([0 if np.isnan(y) else y for y in x])

yearly_perf_summary = bysitebyyear[['model','wind_source','obsgroup','relative_error_pct_of_max']].groupby(['model','wind_source','obsgroup']).\
    agg({ "relative_error_pct_of_max": ["mean","median",p25,p75] }).\
    rename(columns={"<lambda_0>":'25%ile',"<lambda_1>":'75%ile'})
yearly_perf_summary

In [ ]:
yearly_perf_summary.columns = ['_'.join(col) for col in yearly_perf_summary.columns.values]
yearly_perf_summary.reset_index(inplace=True)
yearly_perf_summary.to_csv("04 Error Metrics/bergey_annual_relative_error.csv.bz2",index=False)

#### Performance Drill Down: ANL

In [ ]:
fig = px.box(bysitebymonth.where(bysitebymonth.model == "anl").dropna(),x="month",y="relative_error_pct_of_max",
        facet_col="wind_source",facet_row="obsgroup",
        labels={"relative_error_pct_of_max":"Error (%)","month":"Month"},
        title="Median performance of ANL Model vs. Month of Year (all sites)")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
dc = merged_df[(merged_df["model"] == "anl") & (merged_df["wind_source"] == "wtk") & (merged_df['obsgroup'] == "bldgsonly")].groupby(["tid","month"]).agg({"relative_error_pct_of_max": "median"}).reset_index()
px.imshow(dc.pivot(index="month",columns="tid")["relative_error_pct_of_max"],
          title="Median Error by Site and Month for ANL/WTK",
          color_continuous_scale='Portland')

In [ ]:
dc = merged_df[(merged_df["model"] == "anl") & (merged_df["wind_source"] == "wtk_bc") & (merged_df['obsgroup'] == "bldgsonly")].groupby(["tid","month"]).agg({"relative_error_pct_of_max": "median"}).reset_index()
px.imshow(dc.pivot(index="month",columns="tid")["relative_error_pct_of_max"],
          title="Median Error by Site and Month for ANL/WTK-Bias",
          color_continuous_scale='Portland')

#### Performance Drill Down: Perera

In [ ]:
px.box(bysitebymonth.where(bysitebymonth.model == "perera").dropna(),x="month",y="relative_error_pct_of_max",facet_col="wind_source",facet_col_wrap=3,
        labels={"relative_error_pct":"Median Relative Error (%)","month":"Month"},
        title="Median performance of Perera Model vs. Month of Year (all sites)")

In [ ]:
dc = merged_df[(merged_df["model"] == "perera") & (merged_df["wind_source"] == "wtk")].groupby(["tid","month"]).agg({"relative_error_pct_of_max": "median"}).reset_index()
px.imshow(dc.pivot(index="month",columns="tid")["relative_error_pct_of_max"],
          title="Median Error by Site and Month for Perera/WTK",
          color_continuous_scale='Portland')

In [ ]:
dc = merged_df[(merged_df["model"] == "perera") & (merged_df["wind_source"] == "wtk_bc")].groupby(["tid","month"]).agg({"relative_error_pct_of_max": "median"}).reset_index()
px.imshow(dc.pivot(index="month",columns="tid")["relative_error_pct_of_max"],
          title="Median Error by Site and Month for Perera/WTK-Bias",
          color_continuous_scale='Portland')

#### Performance Drill Down: No Model

In [ ]:
px.box(bysitebymonth.where(bysitebymonth.model == "none").dropna(),x="month",y="relative_error_pct_of_max",facet_col="wind_source",facet_col_wrap=3,
        labels={"relative_error_pct":"Median Relative Error (%)","month":"Month"},
        title="Median performance of No Model vs. Month of Year (all sites)")

In [ ]:
dc = merged_df[(merged_df["model"] == "none") & (merged_df["wind_source"] == "wtk")].groupby(["tid","month"]).agg({"relative_error_pct_of_max": "median"}).reset_index()
px.imshow(dc.pivot(index="month",columns="tid")["relative_error_pct_of_max"],
          title="Median Error by Site and Month for None/WTK",
          color_continuous_scale='Portland')

In [ ]:
dc = merged_df[(merged_df["model"] == "none") & (merged_df["wind_source"] == "wtk_bc")].groupby(["tid","month"]).agg({"relative_error_pct_of_max": "median"}).reset_index()
px.imshow(dc.pivot(index="month",columns="tid")["relative_error_pct_of_max"],
          title="Median Error by Site and Month for None/WTK-Bias",
          color_continuous_scale='Portland')

#### Summary

 - All models produce high errors sometimes, but the bulk of errors (IQR) are within +/- 10%
 - WTK LED 2018 has very high bias, 2019 appears better
 - Bias correction makes a very large difference, especially for WTK LED
 - The Perera suite of models produces extremely high estimates in some cases, which is likely a bug
 - Overall Perera outperforms ANL (!)
 - When resource is not bias corrected, the resource data error dominates, when corrected, the obstacle model error dominates. Residual errors in the resource tend to propagate through the models.
 - Sites 34 and 183 have a high residual bias (resource is overestimated), sites 170 and 28 have low residual bias (resource is underestimated). Curiously, ANL manages to "fix" 183, but not 34. Perera fixes neither. For underestimates, the models seem ill-equipped to fix the problem.
   - Site 34: on southern coast of lake ontario, many trees (wind from SW and SE)
   - Site 183: by lakeland college, large building to the NE, otherwise open (wind primarily from S, sometimes NW)
   - Site 170: jackson college, built environment to the S (wind varies, predominately SW)
   - Site 28: stagecoach rd, rural/suburban but many mature trees around turbine (wind mostly W and NE)

#### Wind Roses

In [ ]:
wtk = pd.read_csv("01 Bergey Turbine Data/wtk.csv.bz2")
wtk.head()

In [ ]:
wtk["ws_rounded"] = wtk["ws"].round()
wtk["wd_rounded"] = 10*(wtk["wd"]/10).astype(int)
dc = wtk.copy()[["tid","wd_rounded","ws_rounded"]]
dc["n"] = 1
dc = dc.groupby(["tid","wd_rounded","ws_rounded"]).sum()
dc = dc.reset_index()
dc['p'] = np.round(100*dc['n']/61368,1)

In [ ]:
for tid in index.tids():
    fig = px.bar_polar(dc[dc["tid"] == tid], r="p", theta="wd_rounded",
                   color="ws_rounded",
                   color_continuous_scale= 'Jet',
                   labels={"ws_rounded": "Windspeed (m/s)","p": "Percent (%)"},
                   title=tid)
    fig.show()